In [55]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import init
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from SSIM_PIL import compare_ssim
from PIL import Image

In [56]:
image1=np.load(r"C:\Users\rohan\Downloads\Miniproject (2)\Miniproject\data\radon\shepp.npy")
print(image1.shape)
#image2=np.load(r"fox.npy")

(512, 180, 1)


In [57]:
def radon(image):
  store = []
  image=image.permute(2,0,1)
  for angle in range(180):
      rotated_image = torchvision.transforms.functional.rotate(image, angle)
      s = torch.sum(rotated_image, 1)
      store.append(s)
  r = torch.stack(store, 1)
  r=r.squeeze(2).permute(2,1,0)
  return r

In [58]:
print(image1.shape)
x,y=np.meshgrid(np.linspace(0,1,512),np.linspace(0,1,512))
x=x.reshape((512*512),1)
y=y.reshape((512*512),1)
pixels=np.hstack((x,y))
pixels=torch.Tensor(pixels)
transforms=torchvision.transforms.ToTensor()
image1=transforms(image1)
print(pixels.numpy().shape)

(512, 180, 1)
(262144, 2)


In [59]:
def psnr(est,gt):
    psnr =-20*np.log10(np.linalg.norm((gt-est).reshape(-1))/(np.sqrt(gt.size)*np.max(gt)))
    return psnr

In [60]:
class Implicit(nn.Module):
    def __init__(self):
        super(Implicit, self).__init__()
        self.linear1=nn.Linear(2,128)
        self.relu1=nn.ReLU()
        self.linear2=nn.Linear(128,256)
        self.linear3=nn.Linear(256,512)
        self.linear4=nn.Linear(512,512)
        self.linear5=nn.Linear(512,256)
        self.linear6=nn.Linear(256,128)
        self.linear7=nn.Linear(128,64)
        self.linear8=nn.Linear(64,32)
        self.linear9=nn.Linear(32,1)
        self.a=nn.Sigmoid()
    def forward(self,x):
        x=self.linear1(x)
        x=self.relu1(x)
        x=self.linear2(x)
        x=self.relu1(x)
        x=self.linear3(x)
        x=self.relu1(x)
        x=self.linear4(x)
        x=self.relu1(x)
        x=self.linear5(x)
        x=self.relu1(x)
        x=self.linear6(x)
        x=self.relu1(x)
        x=self.linear7(x)
        x=self.relu1(x)
        x=self.linear8(x)
        x=self.relu1(x)
        x=self.linear9(x)
        x=self.a(x)
        return x

In [ ]:
implicit=Implicit()
criterion = nn.MSELoss() 
optimizer = torch.optim.Adam(implicit.parameters(),lr = 0.001)
n_epochs=50
x=pixels
train_loss = 0
epochtrack=[]
psnrtrack=[]
simtrack=[]
trainlosstrack=[]
for epoch in range(n_epochs):
    optimizer.zero_grad()
    out=implicit(x)
    out=out.reshape(1,512,512)
    newoutput=radon(out,180,1)
    loss=criterion(newoutput,image1)
    loss.backward()
    optimizer.step()
    trainlosstrack.append(loss.item())
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, loss.item()))
    newoutput=newoutput.reshape(512,180)
    image1=image1.reshape(512,180)
    psnrvalue=psnr(newoutput.detach().numpy(),image1.detach().numpy())
    psnrtrack.append(psnrvalue)
    value = compare_ssim(Image.fromarray((newoutput.detach().numpy()*255).astype(np.uint8)),Image.fromarray((image1.detach().numpy()*255).astype(np.uint8)))
    simtrack.append(value)

In [ ]:
print(trainlosstrack)
plt.plot(psnrtrack)
plt.title('PSNR vs Epochs')
plt.show()
plt.plot(simtrack)
plt.title('SSIM vs Epochs')
plt.show()
out=implicit(pixels)
plt.imshow(out.reshape(512,512,1).detach().numpy())
plt.show()
plt.imshow(image1)
plt.show()
image1=image1.reshape(512*180*1)
out=out.reshape(512*180*1)
print(out.shape)
print(image1.shape)